In [1]:
import numpy as np
import os
%matplotlib qt
import matplotlib.pyplot as pl
import pickle
import qiskit
from qiskit import Aer
import time

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import qiskit_U_1_weak_measurements
import qiskit_evolution_utils

ModuleNotFoundError: No module named 'unitary_sampler'

In [5]:
aa = np.arange(0,32,1).reshape((2,)*5)

In [18]:
bb = aa.reshape((4,4,2))
bb[[(1,0,0),(2,0,0)]]

/var/folders/tm/g65yh1_d5f3gkpxmzywryqwc0000gn/T/ipykernel_69708/3194705934.py:2: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  bb[[(1,0,0),(2,0,0)]]


array([[12, 13],
       [ 0,  1],
       [ 0,  1]])

In [351]:
## Get samples
def get_samples(circ,m_locs,L,T,shots=100):
    results = []
    measurement_data = []
    entropy_data = []
    correlation_data = []
    for s in range(shots):
        # start = time.time()
        job, measurement_array, _ = qiskit_U_1_weak_measurements.simulate_weak_circuit_ancilla(circ,m_locs=m_locs,L=L,T=T,shots=shots)
        results.append(job.result())
        
        measurement_data.append(measurement_array)
        # print("shot_index ", s)
        # print("simulation time ", time.time()-start)
        start = time.time()
        entropy_data.append(qiskit_evolution_utils.job_entropy(job,[0],renyi_index=1))
        # print("entropy calculation time ", time.time()-start)

        # print(entropy_data)
        # start = time.time()
        physical_indices = list((range(L,0,-1)))
        correlation_data.append(qiskit_evolution_utils.job_correlation(job,L,physical_indices=physical_indices))
        # print("correlation calculation time ", time.time()-start)

    return measurement_data, entropy_data, correlation_data




In [352]:
def get_filename(L,T,t_scr,theta,seed,evolution_U_type,scram_U_type,BC,root_direc):
    file_dir = root_direc

    if t_scr==0:
        file_dir = file_dir + 'without_scrambling' 
    else:
        if scram_U_type == 'haar':
            file_dir = file_dir + 'haar_scrambling' 
        elif scram_U_type == 'matchgate':
            file_dir = file_dir + 'matchgate_scrambling' 
    if evolution_U_type == 'haar':
        file_dir = file_dir + '/haar_evolution'
    if evolution_U_type == 'matchgate':
        file_dir = file_dir + '/matchgate_evolution'
    file_dir = file_dir + '/L='+str(L)
    if not os.path.isdir(file_dir):
        os.makedirs(file_dir)
    filename = file_dir+'/T='+str(T)+'_tscr='+str(t_scr)+'_theta='+str(theta)+'_seed='+str(seed)+'_BC='+BC

    return filename

def load_circuit(L,T,t_scr,theta,seed,BC,scram_U_type='haar',evolution_U_type='haar',re_compile=False):

    assert scram_U_type in ['haar','matchgate'], print('scram_U_type can only be either \'haar\' or \'matchgate\'')
    assert evolution_U_type in ['haar','matchgate'], print('evolution_U_type can only be either \'haar\' or \'matchgate\'')


    backend = Aer.get_backend('aer_simulator')
    root_dirc = 'data/circuit_data/'
    filename = get_filename(L=L,T=T,t_scr=t_scr,theta=theta,BC=BC,seed=seed,scram_U_type=scram_U_type,evolution_U_type=evolution_U_type,root_direc=root_dirc)

    if os.path.isfile(filename) and re_compile == False:
        with open(filename,'rb') as f:
            circ, p_locations, total_N_m = pickle.load(f)
    else:
        circ, p_locations, total_N_m  = qiskit_U_1_weak_measurements.get_weak_circuit_ancilla(L=L,T=T,theta=theta,t_scr=t_scr,seed_unitary=seed,BC=BC,scram_U_type=scram_U_type,evolution_U_type=evolution_U_type) # total_N_m is total # of measurements performed
        circ = qiskit.transpile(circ,backend=backend)
        with open(filename,'wb') as f:
            pickle.dump((circ,p_locations,total_N_m),f)
    
    return circ, p_locations, total_N_m


In [22]:
np.where(np.random.uniform(0,1,10)<0.8)[0]

array([0, 2, 3, 4, 5, 6, 7])

In [354]:
L_list = [6,8,10,12]
shots=100
theta_list = np.linspace(0.1,0.6,10)*np.pi/2

root_direc = 'data/simulated_data/'

for theta in theta_list:
    for L in L_list:
        #define parameters
        start = time.time()
        T=2*L
        t_scr = L
        theta = np.round(theta,3)
        scram_U_type = 'haar'
        evolution_U_type = 'haar'
        seed=1
        BC='PBC'
        
        #get data
        circ, m_locs, _ = load_circuit(L=L,T=T,t_scr=t_scr,theta=theta,seed=seed,BC=BC,scram_U_type=scram_U_type,evolution_U_type=evolution_U_type,re_compile=False)
        measurement_data, entropy_data, correlation_data = get_samples(circ,m_locs=m_locs,L=L,T=T,shots=shots)

        #save data
        file_dir = get_filename(L=L,T=T,t_scr=t_scr,theta=theta,BC=BC,seed=seed,scram_U_type=scram_U_type,evolution_U_type=evolution_U_type,root_direc=root_direc)
        if not os.path.isdir(file_dir):
            os.makedirs(file_dir)
        filename = file_dir+'/shots='+str(shots)+'_'+str(np.random.randint(0,1000000000,1))
        with open(filename,'wb') as f:
            pickle.dump((measurement_data,entropy_data,correlation_data),f)
        
        print("L={}, theta={}, time={}".format(L,theta,time.time()-start))



L=6, theta=0.157, time=26.008237838745117
L=8, theta=0.157, time=73.55665016174316
L=10, theta=0.157, time=286.50026988983154
L=12, theta=0.157, time=1418.768061876297
L=6, theta=0.244, time=25.443893909454346
L=8, theta=0.244, time=71.69050288200378
L=10, theta=0.244, time=282.96602988243103
L=12, theta=0.244, time=1400.6072850227356
L=6, theta=0.332, time=24.941197872161865
L=8, theta=0.332, time=71.37021803855896
L=10, theta=0.332, time=281.2730219364166
L=12, theta=0.332, time=1407.7824590206146
L=6, theta=0.419, time=25.307976007461548
L=8, theta=0.419, time=71.50643301010132
L=10, theta=0.419, time=282.4602198600769
L=12, theta=0.419, time=1412.4422509670258
L=6, theta=0.506, time=25.364228010177612
L=8, theta=0.506, time=72.25331711769104
L=10, theta=0.506, time=281.4782190322876
L=12, theta=0.506, time=1404.4518899917603
L=6, theta=0.593, time=25.262434244155884
L=8, theta=0.593, time=71.79574990272522
L=10, theta=0.593, time=284.7586181163788
L=12, theta=0.593, time=1408.66470

In [344]:
print('a={},b={},c={}'.format(1,2,3-3))

a=1,b=2,c=0


In [326]:
[pl.plot(range(14),np.average(np.array(correlation_data)[:,:,0,6],axis=0))]

IndexError: index 6 is out of bounds for axis 3 with size 6

In [329]:
pl.plot(range(13),np.average(np.array(entropy_data),axis=0))

In [90]:
test = np.array([0.        +0.j,         0.65128847-0.27536035j, 0.        +0.j,
 0.        +0.j,         0.        +0.j,         0.        +0.j,
 0.65128847-0.27536035j, 0.        +0.j ]       )

In [91]:
import entanglement

In [94]:
entanglement.renyi_entropy(test,list([0]),2)

1.0000000374693105

In [308]:
rng = np.random.default_rng(seed=1)
rng.integers(0,2,100)

array([0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1])

In [335]:
np.random.randint(0,1000000000,10)

array([317184224, 323746653, 187050997, 424350720, 317085455, 360384100,
       910904913, 977971013, 805359095, 973830962])

In [171]:
import time
time.time

<function time.time>

In [256]:
from functools import reduce
reduce((lambda x,y:x*y),(2,3,5))

30

In [262]:
reduce((lambda x:x),(2,3,4))

TypeError: <lambda>() takes 1 positional argument but 2 were given

## Test to see of initializing scrambled state is faster or not?

In [290]:
L=16
qubits = qiskit.QuantumRegister(L)
circ = qiskit.QuantumCircuit(qubits)
for i in range(0,L,2):
    circ.x(qubits[i])
circ = evolution_utils.scramble(circ,qubits,t_scr=50,seed_scram=200)
circ.save_statevector()
backend = Aer.get_backend('aer_simulator')
circ = qiskit.transpile(circ,backend=backend)
start = time.time()
for s in range(100):
    job = qiskit.execute(circ,backend=backend,shots=1)
print(time.time()-start)

20.926964044570923


In [291]:
qubits2 = qiskit.QuantumRegister(L)
circ2 = qiskit.QuantumCircuit(qubits2)
circ2.initialize(job.result().data()['statevector'],qubits2)
backend = Aer.get_backend('aer_simulator')
circ = qiskit.transpile(circ,backend=backend)

start = time.time()
for s in range(100):
    job = qiskit.execute(circ2,backend=backend,shots=1)
print(time.time()-start)

80.85845494270325


#### Conclusion: Saving the scrambled state has no obvious benefits for large L